# Simulating Neural Governance on cadCAD

## Part 1: Pre-requisites

## Part 2. Module Logic

In [1]:
# Attribute Voting Power to a (user, project) tuple
def user_project_vote_power(uid: UserUUID, 
                            pid: ProjectUUID, 
                            neuron_layers: tuple[dict, callable],
                            initial_votes: float=0.0,
                            print_on_each_layer=False) -> VotingPower:
    """
    Computes a User Vote towards a Project as based on 
    a Feedforward implementation of Neural Governance for a strictly
    sequential network (no layer parallelism).
    """
    current_vote = initial_votes
    if print_on_each_layer:
            print(f"Layer {0}: {current_vote}")
    for i, layer in enumerate(neuron_layers):
        (neurons, layer_aggregator) = layer
        neuron_votes = []
        for (neuron_label, neuron) in neurons.items():
            (oracle_function, weighting_function) = neuron
            raw_neuron_vote = oracle_function(uid, pid, current_vote)
            neuron_votes.append(weighting_function(raw_neuron_vote))
        current_vote = layer_aggregator(neuron_votes)

        if print_on_each_layer:
            print(f"Layer {i+1}: {current_vote}")

    return current_vote



NameError: name 'UserUUID' is not defined

In [ ]:
class ReputationCategory(Enum):
    Tier3 = auto()
    Tier2 = auto()
    Tier1 = auto()
    Uncategorized = auto()

@dataclass
class ReputationAPI():
    status: int
    reputation_category: ReputationCategory

    def get(user_id):
        return ReputationAPI(status=200, 
                             reputation_category=ReputationCategory.Uncategorized)


REPUTATION_SCORE_MAP = {
    ReputationCategory.Tier3: 0.3,
    ReputationCategory.Tier2: 0.2,
    ReputationCategory.Tier1: 0.1,
    ReputationCategory.Uncategorized: 0.0
}

def reputation_score(user_id) -> VotingPower:
    """
    Oracle Module for the Reputation Score
    """
    bonus = 0.0
    api_result = ReputationAPI.get(user_id)
    if api_result.status == 200:
        bonus = REPUTATION_SCORE_MAP[api_result.reputation_category]
    return bonus


@dataclass
class PastVotingAPI():
    status: int
    active_rounds: list[int]

    def get(user_id):
        return PastVotingAPI(status=200, 
                             active_rounds=[2, 3])

ROUND_BONUS_MAP = {
    1: 0.0,
    2: 0.1,
    3: 0.2,
    4: 0.3
}

def prior_voting_score(user_id, project_id, _) -> VotingPower:
    """
    Oracle Module for the Prior Voting Score
    """

    bonus = 0.0
    api_result = PastVotingAPI.get(user_id)
    if api_result.status == 200:
        user_active_rounds = api_result.active_rounds
        for active_round in user_active_rounds:
            bonus += ROUND_BONUS_MAP[active_round]

In [2]:

LAYER_1_AGGREGATOR = lambda lst: sum(lst)
# Take the product of the list
LAYER_2_AGGREGATOR = lambda lst: reduce((lambda x, y: x * y), lst) 

LAYER_1_NEURONS = {
    'one_person_one_vote': (lambda _1, _2, _3: 1, lambda x: x),
    'trust_score': (lambda uid, _1, _2: trust_score(uid),
                    lambda x: x),
    'reputation_score': (lambda uid, _1, _2: reputation_score(uid),
                         lambda x: x)
}

LAYER_2_NEURONS = {
    'power_before_delegation': (lambda _1, _2, prev_vote: prev_vote,
                                lambda x: x),
    'quorum_delegation': (lambda u, p, _: quorum_delegate_result(u, p), 
                          lambda x: x) # Either 0.0 or 1.0
}

NEURAL_GOVERNANCE_LAYERS = [(LAYER_1_NEURONS, LAYER_1_AGGREGATOR),
                            (LAYER_2_NEURONS, LAYER_2_AGGREGATOR)]



## Part 3. cadCAD Simulation

## Part 4. Results